In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
import sklearn.metrics as metrics
from _datetime import datetime

# 数据预处理开始
# 创建特征
column_names1 = ['CONS_NO', 'DT', 'KWH_END', 'KWH_BEGIN', 'KWH']
stolen_electric_analysis_1 = pd.read_csv(r'stolen_electric_analysis_1.csv')
# 替换缺失值
stolen_electric_analysis_output_2 = stolen_electric_analysis_1.replace('nan', np.nan)
# 丢弃缺失值
stolen_electric_analysis_output_2 = stolen_electric_analysis_output_2.dropna(how='any')
# 保存中间结果1-1.csv
stolen_electric_analysis_output_2.to_csv('1-1.csv', index=False)

In [3]:
# 在上表中添加输出有效开始时间和开始时间前一天用电量、有效结束时间和结束时间用电量、总用电量
stolen_electric_analysis_output_2=pd.read_csv('1-1.csv')
stolen_electric_analysis_2=pd.read_csv('stolen_electric_analysis_2.csv')

CONS_NO_USER=stolen_electric_analysis_2['CONS_NO'].values
CONS_NO_USER2=stolen_electric_analysis_output_2['CONS_NO'].values

CONS_NO, START_DATE, START_KWH_READING1, END_DATE, END_KWH_READING, SUM_KWH = [], [], [], [], [], []
for name in CONS_NO_USER:
    if name in CONS_NO_USER2:
        CONS_NO.append(name)
        _group=stolen_electric_analysis_output_2[stolen_electric_analysis_output_2['CONS_NO']==name]
        s1, s2, s3 = _group['DT'].values, _group['KWH_BEGIN'].values, _group['KWH_END'].values
        START_DATE.append(s1[0])
        START_KWH_READING1.append(s2[0])
        END_DATE.append(s1[-1])
        END_KWH_READING.append(s3[-1])
        SUM_KWH.append(round(float(s3[-1] - s2[0]), 2))

stolen_electric_analysis_output_3 = pd.DataFrame(
    {'CONS_NO': CONS_NO, 'START_DATE': START_DATE, 'TOTAL_START_KWH': START_KWH_READING1, 'END_DATE': END_DATE,
     'TOTAL_END_KWH': END_KWH_READING, 'SUM_KWH': SUM_KWH})
stolen_electric_analysis_output_3.to_csv('1-2.csv', index=False)

8910
第三问结果行列数 (1013, 6)


In [7]:
# 训练模型
stolen_electric_analysis_2 = pd.read_csv(r'stolen_electric_analysis_2.csv')
stolen_electric_analysis_output_3 = pd.read_csv(r'1-2.csv')
train_data = pd.merge(stolen_electric_analysis_output_3, stolen_electric_analysis_2, left_on="CONS_NO", right_on='CONS_NO')
add_ = train_data[train_data['CHK_STATE'] == 1]

for _ in range(0, 9):
    train_data = pd.concat([train_data, add_])
lsvc = LinearSVC()
lsvc.fit(train_data[['SUM_KWH']], train_data[['CHK_STATE']])

# 构建预测值
stolen_electric_analysis_output_2 = pd.read_csv('1-1.csv')
stolen_electric_analysis_3 = pd.read_csv(r'stolen_electric_analysis_3.csv')
CONS_NO_USER = stolen_electric_analysis_3['CONS_NO'].values
CONS_NO_USER2 = stolen_electric_analysis_output_2['CONS_NO'].values
CONS_NO, START_DATE, START_KWH_READING1, END_DATE, END_KWH_READING, SUM_KWH = [], [], [], [], [], []

for name in CONS_NO_USER:
    if name in CONS_NO_USER2:
        CONS_NO.append(name)
        _group=stolen_electric_analysis_output_2[stolen_electric_analysis_output_2['CONS_NO']==name]
        s1, s2, s3 = _group['DT'].values, _group['KWH_BEGIN'].values, _group['KWH_END'].values
        START_DATE.append(s1[0])
        START_KWH_READING1.append(s2[0])
        END_DATE.append(s1[-1])
        END_KWH_READING.append(s3[-1])
        SUM_KWH.append(round(float(s3[-1] - s2[0]), 2))

test_data_X=pd.DataFrame(
    {'CONS_NO': CONS_NO, 'START_DATE': START_DATE, 'TOTAL_START_KWH': START_KWH_READING1, 'END_DATE': END_DATE,
     'TOTAL_END_KWH': END_KWH_READING, 'SUM_KWH': SUM_KWH})
stolen_electric_analysis_3_answer = pd.read_csv(r'stolen_electric_analysis_3_answer.csv')
test_data = pd.merge(test_data_X, stolen_electric_analysis_3_answer, left_on="CONS_NO", right_on='CONS_NO')

X_test = test_data[['SUM_KWH']]  # pd.DataFrame({'SUM_KWH':SUM_KWH})
y_test = test_data[['CHK_STATE']]
_y = lsvc.predict(test_data[['SUM_KWH']])

stolen_electric_analysis_output_4 = pd.DataFrame({'CONS_NO': test_data['CONS_NO'].values, 'CHK_STATE': _y})

print(metrics.classification_report(y_test, lsvc.predict(X_test)))

D:\anaconda\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

           0       0.62      0.24      0.34        34
           1       0.61      0.89      0.72        45

    accuracy                           0.61        79
   macro avg       0.61      0.56      0.53        79
weighted avg       0.61      0.61      0.56        79

